# Lab: Working with Entra ID Privileged Identity Management (PIM)

In this lab, you will learn how to work with Microsoft Entra ID Privileged Identity Management (PIM) using Microsoft Graph PowerShell SDK. PIM is a service that enables you to manage, control, and monitor access to important resources in your organization.

## Prerequisites

To successfully complete this lab, make sure you have the following:

- Microsoft Graph PowerShell SDK installed
- Microsoft Entra ID P2 or Enterprise Mobility + Security (EMS) E5 license
- Appropriate permissions with the Privileged Role Administrator role
- For this lab, the `RoleManagement.ReadWrite.Directory` delegated permission is required

## Lab Overview

In this lab, you will:

1. Connect to Microsoft Graph with appropriate permissions
2. Find available directory roles and role templates
3. Grant an eligible role assignment to a user
4. Extend the duration of an eligible role assignment
5. Activate a role assignment
6. Deactivate an active role assignment
7. Remove an eligible role assignment

## Task 1: Connect to Microsoft Graph

First, we need to connect to Microsoft Graph with the appropriate permissions. For managing PIM, we need the `RoleManagement.ReadWrite.Directory` scope.

> **Note**: When prompted to consent, select "Consent on behalf of your organization" before accepting in the sign-in dialog box.

In [ ]:
# Connect to Microsoft Graph with required permissions
Connect-MgGraph -Scopes 'RoleManagement.ReadWrite.Directory'

## Task 2: Find the User and Available Roles

Now we need to identify the user who will receive the role assignment and learn what roles are available in the system.

In [ ]:
# Get the user information - replace Diego with another user in your tenant
# When you pick a user, reset his/her password
$User = Get-MgUser -UserId 'diegos@M365x33461626.onmicrosoft.com'
$principalId = $User.Id

# Display the user details to confirm
$User | Select-Object DisplayName, UserPrincipalName, Id

### Finding Available Directory Roles

Let's examine what directory roles are currently available in your Entra ID tenant.

In [ ]:
# List currently available directory roles
Get-MgDirectoryRole | Select-Object DisplayName, Id | Sort-Object DisplayName

### Finding All Potential Directory Role Templates

According to the Microsoft Graph API documentation, when assigning a role using the Azure portal, the role activation step is implicitly done on the admin's behalf. To get the full list of roles that are available in Entra ID (including those that haven't been activated yet), use the `Get-MgDirectoryRoleTemplate` cmdlet.

In [ ]:
# List all potential directory role templates
Get-MgDirectoryRoleTemplate | Select-Object DisplayName, Id | Sort-Object DisplayName

## Task 3: Assign an Eligible Role

Now we will assign an eligible `Security Reader` role to our user. This makes the user eligible to activate the role when needed but doesn't grant the permissions right away.

In [ ]:
# Get the Security Reader role definition ID
$RoleDefinitionId = (Get-MgDirectoryRoleTemplate | Where-Object DisplayName -Match 'Security Reader').Id

# Create parameters for eligible role assignment
$params = @{
    'PrincipalId'      = $principalId
    'RoleDefinitionId' = $RoleDefinitionId
    'Justification'    = 'Add eligible assignment'
    'DirectoryScopeId' = '/'  # '/' represents the entire directory
    'Action'           = 'AdminAssign'
    'ScheduleInfo'     = @{
        'StartDateTime' = Get-Date
        'Expiration'    = @{
            'Type'     = 'AfterDuration'
            'Duration' = 'PT1H'  # ISO 8601 duration format - 1 hour
        }
    }
}

# Create the eligible role assignment
New-MgRoleManagementDirectoryRoleEligibilityScheduleRequest -BodyParameter $params | Format-List *

## Task 4: Verify the Eligible Assignment

Let's verify that our user now has an eligible role assignment.

In [ ]:
# Get the eligible role assignments for our user
Get-MgRoleManagementDirectoryRoleEligibilityScheduleInstance -Filter "principalId eq '$principalId'" | Format-List

## Task 5: Extend an Eligible Assignment

If we need to extend the eligible assignment for a longer period, we can use the `AdminExtend` action.

In [ ]:
# Create parameters to extend the eligible assignment
$params = @{
    'PrincipalId'      = $principalId
    'RoleDefinitionId' = $RoleDefinitionId
    'Justification'    = 'Extend eligible assignment'
    'DirectoryScopeId' = '/'
    'Action'           = 'AdminExtend'
    'ScheduleInfo'     = @{
        'StartDateTime' = Get-Date
        'Expiration'    = @{
            'Type'        = 'AfterDateTime'
            'EndDateTime' = (Get-Date).AddDays(1)  # Extend for 1 day
        }
    }
}

# Extend the eligible role assignment
New-MgRoleManagementDirectoryRoleEligibilityScheduleRequest -BodyParameter $params |
    Format-List Id, Status, Action, DirectoryScopeId, RoleDefinitionId, Justification, PrincipalId

## Task 6: Self-Activate a Role (As the User)

First, we need to connect as the user who has the eligible role assignment.

In [ ]:
# Disconnect from the current session
Disconnect-MgGraph

# Connect to Microsoft Graph as the user (in a real scenario)
# Note: You'll need to sign in as the user when prompted
Connect-MgGraph -ContextScope Process

In [ ]:
# Get the user information - replace Diego with another user in your tenant
$User = Get-MgUser -UserId 'diegos@M365x33461626.onmicrosoft.com'
$principalId = $User.Id
$RoleDefinitionId = (Get-MgDirectoryRoleTemplate | Where-Object DisplayName -Match 'Security Reader').Id

# Create parameters for role activation
$params = @{
    'PrincipalId'      = $principalId
    'RoleDefinitionId' = $RoleDefinitionId
    'Justification'    = 'Activate assignment'
    'DirectoryScopeId' = '/'
    'Action'           = 'SelfActivate'
    'ScheduleInfo'     = @{
        'StartDateTime' = Get-Date
        'Expiration'    = @{
            'Type'     = 'AfterDuration'
            'Duration' = 'PT1H'  # Activate for 1 hour
        }
    }
}

# Activate the role
New-MgRoleManagementDirectoryRoleAssignmentScheduleRequest -BodyParameter $params |
    Format-List Id, Status, Action, DirectoryScopeId, RoleDefinitionId, Justification, PrincipalId

## Task 7: Verify Active Role Assignments

Let's verify that the role is now active for our user.

In [ ]:
# Get the active role assignments for our user
Get-MgRoleManagementDirectoryRoleAssignmentScheduleInstance -Filter "principalId eq '$principalId'" | Format-List

## Task 8: Deactivate a Role

When the user is finished with the elevated privileges, they can deactivate the role.

In [ ]:
# Create parameters for role deactivation
$params = @{
    'PrincipalId'      = $principalId
    'RoleDefinitionId' = $RoleDefinitionId
    'Justification'    = 'Deactivate assignment'
    'DirectoryScopeId' = '/'
    'Action'           = 'SelfDeactivate'
}

# Deactivate the role
New-MgRoleManagementDirectoryRoleAssignmentScheduleRequest -BodyParameter $params |
    Format-List Id, Status, Action, DirectoryScopeId, RoleDefinitionId, Justification, PrincipalId

## Task 9: Remove an Eligible Assignment

Finally, as an administrator, you can remove the eligible assignment completely. This can be done either by the administrator or the user.

In [ ]:
# Create parameters to remove the eligible assignment
$params = @{
    'PrincipalId'      = $principalId
    'RoleDefinitionId' = $RoleDefinitionId
    'Justification'    = 'Remove eligible assignment'
    'DirectoryScopeId' = '/'
    'Action'           = 'AdminRemove'
}

# Remove the eligible role assignment
New-MgRoleManagementDirectoryRoleEligibilityScheduleRequest -BodyParameter $params |
    Format-List Id, Status, Action, DirectoryScopeId, RoleDefinitionId, Justification, PrincipalId

## Summary

In this lab, you learned how to:

- Connect to Microsoft Graph with appropriate permissions for PIM operations
- View available directory roles and role templates
- Assign an eligible role to a user
- Extend the duration of an eligible role assignment
- Activate a role as the user
- Deactivate an active role assignment
- Remove an eligible role assignment

These operations are the fundamental building blocks for implementing Just-In-Time (JIT) access and the principle of least privilege in your organization using Microsoft Entra ID Privileged Identity Management.

## Challenge Exercises

1. Assign multiple eligible roles to a user
2. Create a script that automates checking for expired eligible assignments
3. Create a schedule that automatically deactivates roles after business hours